In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
#pd.set_option('display.max_rows', None)
df_train=pd.read_csv("C:/Users/maryshe/Documents/Data Science_Inernship Worksheets/Malignant-Comments-Classifier-Project--1---1-/Malignant Comments Classifier Project/train.csv")
df_test=pd.read_csv("C:/Users/maryshe/Documents/Data Science_Inernship Worksheets/Malignant-Comments-Classifier-Project--1---1-/Malignant Comments Classifier Project/test.csv")
df_train.head()

,id,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
print(df_test.shape)
print(df_train.shape)

(153164, 2)
(159571, 8)


In [4]:
df_test

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [5]:
#Merge the 2 dataframes test and train
df = df_train.merge(df_test, how='outer')

In [6]:
#drop the id column
df_train=df_train.drop(columns='id')
df_test=df_test.drop(columns='id')
df=df.drop(columns='id')

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   comment_text      159571 non-null  object
 1   malignant         159571 non-null  int64 
 2   highly_malignant  159571 non-null  int64 
 3   rude              159571 non-null  int64 
 4   threat            159571 non-null  int64 
 5   abuse             159571 non-null  int64 
 6   loathe            159571 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 8.5+ MB


## EDA

In [8]:
#Understanding what is the data available in df
int_features = df_train.select_dtypes(include=[np.int64])
#frequency of categories
for col in int_features:
    print(df_train[col].value_counts())
    

0    144277
1     15294
Name: malignant, dtype: int64
0    157976
1      1595
Name: highly_malignant, dtype: int64
0    151122
1      8449
Name: rude, dtype: int64
0    159093
1       478
Name: threat, dtype: int64
0    151694
1      7877
Name: abuse, dtype: int64
0    158166
1      1405
Name: loathe, dtype: int64


In [9]:
##These all look imbalanced data labels!!
print("malignant ratio=",len(df_train[df_train['malignant']==1])/len(df_train.malignant)*100,'%')
print("highly_malignant' ratio=",len(df_train[df_train['highly_malignant']==1])/len(df_train.highly_malignant)*100,'%')
print("rude=",len(df_train[df_train['rude']==1])/len(df_train.rude)*100,'%')
print("threat=",len(df_train[df_train['threat']==1])/len(df_train.threat)*100,'%')
print("abuse=",len(df_train[df_train['abuse']==1])/len(df_train.abuse)*100,'%')
print("loathe=",len(df_train[df_train['loathe']==1])/len(df_train.loathe)*100,'%')

malignant ratio= 9.584448302009765 %
highly_malignant' ratio= 0.9995550569965721 %
rude= 5.2948217407925 %
threat= 0.2995531769557125 %
abuse= 4.936360616904074 %
loathe= 0.8804858025581089 %


In [10]:
## Convert all the comments 
df['comment_text']=df['comment_text'].str.lower()
df_train['comment_text']=df_train['comment_text'].str.lower()
df_test['comment_text']=df_test['comment_text'].str.lower()

## Claening the columns

In [11]:
## Remove the punctuations

import string
print(string.punctuation)
# Define the function to remove the punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
# Apply to the DF series
df_train['comment_text']= df_train['comment_text'].apply(remove_punctuations)
df_test['comment_text']= df_test['comment_text'].apply(remove_punctuations)


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [12]:
# removing stop words
# Import stopwords with scikit-learn
from sklearn.feature_extraction import text
stop = text.ENGLISH_STOP_WORDS


df_train['comment_text'] = df_train['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df_train['comment_text'])
df_test['comment_text'] = df_test['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df_test['comment_text'])

0         explanation edits username hardcore metallica ...
1         daww matches background colour im seemingly st...
2         hey man im really trying edit war just guy con...
3         make real suggestions improvement wondered sec...
4                       sir hero chance remember page thats
                                ...                        
159566    second time asking view completely contradicts...
159567           ashamed horrible thing talk page 128611993
159568    spitzer umm theres actual article prostitution...
159569      looks like actually speedy version deleted look
159570    really dont think understand came idea bad rig...
Name: comment_text, Length: 159571, dtype: object
0         yo bitch ja rule succesful youll whats hating ...
1                                        rfc title fine imo
2                             sources zawe ashton lapland —
3         look source information updated correct form g...
4                            dont anonymously edit

In [13]:
# Removing all digits
df_train['comment_text'] = df_train['comment_text'].str.replace(r'\d+(\.\d+)?', 'numbr')
df_test['comment_text'] = df_test['comment_text'].str.replace(r'\d+(\.\d+)?', 'numbr')

In [14]:
#Lemmatization to normalize the words in different form 

from nltk.stem import WordNetLemmatizer

lem=WordNetLemmatizer()
df_train['comment_text'] = df_train['comment_text'].apply(lambda x: ' '.join(
 lem.lemmatize(t) for t in x.split()))

In [15]:
#If my Lemmatization worded
df_train['comment_text']

0         explanation edits username hardcore metallica ...
1         daww match background colour im seemingly stuc...
2         hey man im really trying edit war just guy con...
3         make real suggestion improvement wondered sect...
4                       sir hero chance remember page thats
                                ...                        
159566    second time asking view completely contradicts...
159567               ashamed horrible thing talk page numbr
159568    spitzer umm there actual article prostitution ...
159569       look like actually speedy version deleted look
159570    really dont think understand came idea bad rig...
Name: comment_text, Length: 159571, dtype: object

from nltk import word_tokenize, pos_tag
tok_and_tag = lambda x: pos_tag(word_tokenize(x))
df_test['comment_text']= df_test['comment_text'].apply(tok_and_tag)

import nltk
nltk.download('averaged_perceptron_tagger')

df_test['comment_text']

 #tag_dict = {"J": wordnet.ADJ,
                #"N": wordnet.NOUN,
                #"V": wordnet.VERB,
                #"R": wordnet.ADV}

## Visualization of the comments how therer are classified

## redefining variables for visualization purpose
df_test_vis=df_test
df_train_vis=df_train

import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag
tok_and_tag = lambda x: pos_tag(word_tokenize(x))
df_train_vis['comment_text']= df_train_vis['comment_text'].apply(tok_and_tag)

df_train_vis['comment_text'].unique

In [16]:
#df_test['comment_text']

 #tag_dict = {"J": wordnet.ADJ,
                #"N": wordnet.NOUN,
                #"V": wordnet.VERB,
                #"R": wordnet.ADV}

#Getting sense of loud words which are offensive
from wordcloud import WordCloud

hams = df_test_vis['comment_text']==JJ
spam_cloud = WordCloud(width=600,height=400,background_color='black',max_words=50).generate(' '.join(hams))
plt.figure(figsize=(10,8),facecolor='k')
plt.imshow(spam_cloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

## Model preparing

In [17]:
# All the comments are summed up, If the comment is very bad it would give a higher score or less score
cols_target =['malignant','highly_malignant','rude','threat','abuse','loathe']

df_train['bad'] =df_train[cols_target].sum(axis =1)
print(df_train['bad'].value_counts())
df_train

0    143346
1      6360
3      4209
2      3480
4      1760
5       385
6        31
Name: bad, dtype: int64


,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,bad
0,explanation edits username hardcore metallica ...,0,0,0,0,0,0,0
1,daww match background colour im seemingly stuc...,0,0,0,0,0,0,0
2,hey man im really trying edit war just guy con...,0,0,0,0,0,0,0
3,make real suggestion improvement wondered sect...,0,0,0,0,0,0,0
4,sir hero chance remember page thats,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,second time asking view completely contradicts...,0,0,0,0,0,0,0
159567,ashamed horrible thing talk page numbr,0,0,0,0,0,0,0
159568,spitzer umm there actual article prostitution ...,0,0,0,0,0,0,0
159569,look like actually speedy version deleted look,0,0,0,0,0,0,0


In [18]:
cols_target =['malignant','highly_malignant','rude','threat','abuse','loathe']

df_train['bad'] =df_train[cols_target].sum(axis =1)
print(df_train['bad'].value_counts())
df_train

0    143346
1      6360
3      4209
2      3480
4      1760
5       385
6        31
Name: bad, dtype: int64


,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,bad
0,explanation edits username hardcore metallica ...,0,0,0,0,0,0,0
1,daww match background colour im seemingly stuc...,0,0,0,0,0,0,0
2,hey man im really trying edit war just guy con...,0,0,0,0,0,0,0
3,make real suggestion improvement wondered sect...,0,0,0,0,0,0,0
4,sir hero chance remember page thats,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,second time asking view completely contradicts...,0,0,0,0,0,0,0
159567,ashamed horrible thing talk page numbr,0,0,0,0,0,0,0
159568,spitzer umm there actual article prostitution ...,0,0,0,0,0,0,0
159569,look like actually speedy version deleted look,0,0,0,0,0,0,0


In [19]:
df_train['bad'].unique()

array([0, 4, 1, 3, 2, 5, 6], dtype=int64)

In [20]:
#Vectorized to check the frequency of 10000  frequent words_df_train
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vec = TfidfVectorizer(max_features = 1000, stop_words='english')
features = tf_vec.fit_transform(df_train['comment_text'])
x = features

In [21]:
#Vectorized to check the frequency of 10000  frequent words_df_test
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vec = TfidfVectorizer(max_features = 1000, stop_words='english')
features = tf_vec.fit_transform(df_test['comment_text'])
x_test_actual= features

In [22]:
df_train=df_train.drop(columns=['malignant','highly_malignant','rude','threat','abuse','loathe'])

In [23]:
df_train

,comment_text,bad
0,explanation edits username hardcore metallica ...,0
1,daww match background colour im seemingly stuc...,0
2,hey man im really trying edit war just guy con...,0
3,make real suggestion improvement wondered sect...,0
4,sir hero chance remember page thats,0
...,...,...
159566,second time asking view completely contradicts...,0
159567,ashamed horrible thing talk page numbr,0
159568,spitzer umm there actual article prostitution ...,0
159569,look like actually speedy version deleted look,0


In [24]:
y=df_train['bad']

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25, random_state=100)

In [26]:
from sklearn.linear_model import LogisticRegression
LG=LogisticRegression()
LG.fit(x_train,y_train)
y_pred=LG.predict(x_test)

In [27]:
### Classification report- Which gives accuracy, precision, F1 score
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test,y_pred,digits=2))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97     35822
           1       0.31      0.11      0.17      1601
           2       0.35      0.09      0.14       893
           3       0.48      0.52      0.50      1040
           4       0.45      0.22      0.29       431
           5       0.33      0.01      0.02        98
           6       0.00      0.00      0.00         8

    accuracy                           0.92     39893
   macro avg       0.41      0.28      0.30     39893
weighted avg       0.89      0.92      0.90     39893

[[35694   106     8    14     0     0     0]
 [ 1290   183    42    82     4     0     0]
 [  456   136    77   215     9     0     0]
 [  258   122    62   536    61     1     0]
 [   60    34    20   223    94     0     0]
 [    9     3    10    35    40     1     0]
 [    1     0     0     4     2     1     0]]


## As the classification sccuracy for lower number classes  is less  model is not working well.

SMOTE would not work to oversampke the data, Let try putting all the bad comments into one class


In [28]:
df_train['bad'] = df_train['bad'].replace({2:1,3:1, 4:1, 5:1, 6:1},regex=True)

In [29]:
df_train['bad']=y

In [30]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25, random_state=100)

In [31]:
from sklearn.linear_model import LogisticRegression
LG=LogisticRegression()
LG.fit(x_train,y_train)
y_pred=LG.predict(x_test)

In [32]:
### Classification report- Which gives accuracy, precision, F1 score
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test,y_pred,digits=2))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     35822
           1       0.90      0.55      0.68      4071

    accuracy                           0.95     39893
   macro avg       0.93      0.77      0.83     39893
weighted avg       0.95      0.95      0.94     39893

[[35574   248]
 [ 1848  2223]]


In [33]:
#Model looks promising, Let use with test data


y_pred_actual=LG.predict(x_test_actual)

In [34]:
y_pred_actual

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [35]:
#building a csv
np.savetxt('test_actual.csv', y_pred_actual, delimiter=',')